In [1]:
def df_from_redshift(sql, engine):
    with engine.connect().execution_options(autocommit=True) as conn:
        query = conn.execute(text(sql))
        df = pd.DataFrame(query.fetchall())
    return df

## sies-matriculados

* Matriculados por tipo de institución

In [ ]:
sql_insumo = '''SELECT periodo,clasificacion_institucion_nivel_3,sum(coalesce(total_matriculados,0)) as total_insumo 
           FROM sies.sies_mat_h_2024
           GROUP BY clasificacion_institucion_nivel_3,periodo
           ORDER BY periodo,clasificacion_institucion_nivel_3;'''

df_insumo = df_from_redshift(sql_insumo, engine_insumo)
df_insumo

In [ ]:
sql_cubo = '''SELECT a.periodo,b.clasificacion_institucion_nivel_3,sum(coalesce(total_matriculados,0)) as total_cubo
              FROM programas.agg_programas as a
              LEFT JOIN programas.dim_programas as b
              ON a.codigo_unico=b.codigo_unico AND a.periodo=b.periodo
              WHERE a.existencia_sies_mat=1
              GROUP BY b.clasificacion_institucion_nivel_3,a.periodo;'''

df_cubo = df_from_redshift(sql_cubo, engine_cubo)
df_cubo

In [ ]:
df = df_insumo.merge(df_cubo)

df['diferencias'] = df['total_insumo'] - df['total_cubo']

df

In [ ]:
df.loc[(df.loc[:,'diferencias']!= 0.)]

In [6]:
#df.to_csv('matriculados_tipo_inst.csv',index=False)
df.loc[(df.loc[:,'diferencias']!= 0.)].to_csv('sies_matriculados_tipo_inst.csv',index=False)

* Matriculados por código de Institución

In [ ]:
sql_insumo = '''SELECT periodo,codigo_institucion,sum(total_matriculados) as total_insumo 
                FROM sies.sies_mat_h_2024
                GROUP BY codigo_institucion,periodo
                ORDER BY periodo,codigo_institucion;'''

df_insumo = df_from_redshift(sql_insumo, engine_insumo)
df_insumo

In [ ]:
sql_cubo = '''SELECT a.periodo,b.codigo_institucion,sum(a.total_matriculados) as total_cubo
              FROM programas.agg_programas as a
              LEFT JOIN programas.dim_programas as b
              ON a.codigo_unico=b.codigo_unico AND a.periodo=b.periodo
              WHERE a.existencia_sies_mat=1
              GROUP BY b.codigo_institucion,a.periodo;'''

df_cubo = df_from_redshift(sql_cubo, engine_cubo)
df_cubo

In [ ]:
df = df_insumo.merge(df_cubo)

df['diferencias'] = df['total_insumo'] - df['total_cubo']

df

In [ ]:
df.loc[(df.loc[:,'diferencias']!= 0.)]

In [12]:
#df.to_csv('matriculados_cod_inst.csv',index=False)
df.loc[(df.loc[:,'diferencias']!= 0.)].to_csv('sies_matriculados_cod_inst.csv',index=False)

* Matriculados por región

In [ ]:
sql_insumo = '''SELECT region_sede,periodo,sum(total_matriculados) as total_insumo 
                FROM sies.sies_mat_h_2023
                GROUP BY region_sede,periodo
                ORDER BY 2,1;'''

df_insumo = df_from_redshift(sql_insumo, engine_insumo)
df_insumo

In [ ]:
sql_cubo = '''SELECT a.periodo,c.nombre_region,sum(a.total_matriculados) as total_cubo
              FROM programas.agg_programas as a
              LEFT JOIN transversal.dim_region as b
              ON a.codigo_unico=b.codigo_unico AND a.periodo=b.periodo
              GROUP BY b.codigo_institucion,a.periodo;'''

df_cubo = df_from_redshift(sql_cubo, engine_cubo)
df_cubo

## sies-titulados

* titulados por tipo de institución

In [ ]:
sql_insumo = '''SELECT periodo,clasificacion_institucion_nivel_3,sum(total_titulados) as total_insumo 
           FROM sies.sies_tit_h_2023
           GROUP BY clasificacion_institucion_nivel_3,periodo
           ORDER BY periodo,clasificacion_institucion_nivel_3;'''

df_insumo = df_from_redshift(sql_insumo, engine_insumo)
df_insumo

In [ ]:
sql_cubo = '''SELECT a.periodo,b.clasificacion_institucion_nivel_3,sum(a.total_titulados) as total_cubo
              FROM programas.agg_programas as a
              LEFT JOIN programas.dim_programas as b
              ON a.codigo_unico=b.codigo_unico AND a.periodo=b.periodo
              WHERE a.existencia_sies_tit=1
              GROUP BY b.clasificacion_institucion_nivel_3,a.periodo;'''

df_cubo = df_from_redshift(sql_cubo, engine_cubo)
df_cubo

In [ ]:
df = df_insumo.merge(df_cubo)

df['diferencias'] = df['total_insumo'] - df['total_cubo']

df

In [ ]:
df.loc[(df.loc[:,'diferencias']!= 0.)]

In [15]:
#df.to_csv('titulados_tipo_inst.csv',index=False)
df.loc[(df.loc[:,'diferencias']!= 0.)].to_csv('sies_titulados_tipo_inst.csv',index=False)

* Titulados por código de Institución

In [ ]:
sql_insumo = '''SELECT periodo,codigo_institucion,sum(total_titulados) as total_insumo 
           FROM sies.sies_tit_h_2023
           GROUP BY codigo_institucion,periodo
           ORDER BY periodo,codigo_institucion;'''

df_insumo = df_from_redshift(sql_insumo, engine_insumo)
df_insumo

In [ ]:
sql_cubo = '''SELECT a.periodo,b.codigo_institucion,sum(a.total_titulados) as total_cubo
              FROM programas.agg_programas as a
              LEFT JOIN programas.dim_programas as b
              ON a.codigo_unico=b.codigo_unico AND a.periodo=b.periodo
              WHERE a.existencia_sies_tit=1
              GROUP BY b.codigo_institucion,a.periodo;'''

df_cubo = df_from_redshift(sql_cubo, engine_cubo)
df_cubo

In [ ]:
df = df_insumo.merge(df_cubo)

df['diferencias'] = df['total_insumo'] - df['total_cubo']

df

In [ ]:
df.loc[(df.loc[:,'diferencias']!= 0.)]

In [23]:
#df.to_csv('titulados_cod_inst.csv',index=False)
df.loc[(df.loc[:,'diferencias']!= 0.)].to_csv('sies_titulados_cod_inst.csv',index=False)

* Titulados por región

In [ ]:
sql_cubo = '''SELECT region_sede,periodo,sum(total_titulados) as total_cubo 
           FROM sies.sies_tit_h_2023
           GROUP BY region_sede,periodo
           ORDER BY 2,1;'''

df_rs = df_from_redshift(sql_cubo, engine_cubo)
df_rs

In [ ]:
df_stage = df_stage.sort_values(['region','anio'])
df_rs = df_rs.sort_values(['region_sede','periodo'])

df_stage['total_cubo'] = df_rs['total_cubo'].values
df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['total_titulados'].values

print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

## sies-oferta-académica

* cantidad programas por tipo de institucion

In [ ]:
sql_insumo = '''SELECT periodo,clasificacion_institucion_nivel_3,count(cod_unico) as total_insumo 
           FROM sies.sies_oa_h_2024
           GROUP BY clasificacion_institucion_nivel_3,periodo
           ORDER BY periodo,clasificacion_institucion_nivel_3;'''

df_insumo = df_from_redshift(sql_insumo, engine_insumo)
df_insumo

In [ ]:
sql_cubo = '''SELECT a.periodo,b.clasificacion_institucion_nivel_3,count(a.codigo_unico) as total_cubo
              FROM programas.agg_programas as a
              LEFT JOIN programas.dim_programas as b
              ON a.codigo_unico=b.codigo_unico AND a.periodo=b.periodo
              WHERE a.existencia_sies_oa=1
              GROUP BY b.clasificacion_institucion_nivel_3,a.periodo;'''

df_cubo = df_from_redshift(sql_cubo, engine_cubo)
df_cubo

In [ ]:
df = df_insumo.merge(df_cubo)

df['diferencias'] = df['total_insumo'] - df['total_cubo']

df

In [ ]:
df.loc[(df.loc[:,'diferencias']!= 0.)]

In [28]:
#df.to_csv('programas_tipo_inst.csv',index=False)
df.loc[(df.loc[:,'diferencias']!= 0.)].to_csv('sies_programas_tipo_inst.csv',index=False)

* Cantidad de programas por código de institución

In [ ]:
sql_insumo = '''SELECT periodo,codigo_institucion,count(cod_unico) as total_insumo 
           FROM sies.sies_oa_h_2024
           GROUP BY codigo_institucion,periodo
           ORDER BY periodo,codigo_institucion;'''

df_insumo = df_from_redshift(sql_insumo, engine_insumo)
df_insumo

In [ ]:
sql_cubo = '''SELECT a.periodo,b.codigo_institucion,count(a.codigo_unico) as total_cubo
              FROM programas.agg_programas as a
              LEFT JOIN programas.dim_programas as b
              ON a.codigo_unico=b.codigo_unico AND a.periodo=b.periodo
              WHERE a.existencia_sies_oa=1
              GROUP BY b.codigo_institucion,a.periodo;'''

df_cubo = df_from_redshift(sql_cubo, engine_cubo)
df_cubo

In [ ]:
df = df_insumo.merge(df_cubo)

df['diferencias'] = df['total_insumo'] - df['total_cubo']

df

In [ ]:
df.loc[(df.loc[:,'diferencias']!= 0.)]

In [33]:
#df.to_csv('programas_cod_inst.csv',index=False)
df.loc[(df.loc[:,'diferencias']!= 0.)].to_csv('sies_programas_cod_inst.csv',index=False)

* Cantidad de programas por región

In [ ]:
sql_cubo = '''SELECT region_sede,periodo,count(cod_unico) as total_cubo 
           FROM sies.sies_oa_h_2024
           GROUP BY region_sede,periodo
           ORDER BY periodo,region_sede;'''

df_rs = df_from_redshift(sql_cubo, engine_cubo)
df_rs

In [ ]:
df_stage = df_stage.sort_values(['region_sede','anio'])
df_rs = df_rs.sort_values(['region_sede','periodo'])

df_stage['total_cubo'] = df_rs['total_cubo'].values
df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['codigo_unico'].values

print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])